In [300]:
import pandas as pd
from azure.cognitiveservices.language.luis.authoring import LUISAuthoringClient
from azure.cognitiveservices.language.luis.authoring.models import ApplicationCreateObject
from azure.cognitiveservices.language.luis.runtime import LUISRuntimeClient
from msrest.authentication import CognitiveServicesCredentials
from functools import reduce
import re

import json, time, uuid, random

In [2]:
dataset = pd.read_json("D:/Data OC/P10/frames/frames.json")

# Discovering the structure

In [8]:
dataset['turns'][0][0]['text']

"I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700."

In [9]:
dataset['turns'][0][0]['labels']['frames']

[{'info': {'intent': [{'val': 'book', 'negated': False}],
   'budget': [{'val': '1700.0', 'negated': False}],
   'dst_city': [{'val': 'Atlantis', 'negated': False}],
   'or_city': [{'val': 'Caprica', 'negated': False}],
   'str_date': [{'val': 'august 13', 'negated': False}],
   'n_adults': [{'val': '8', 'negated': False}]},
  'frame_id': 1,
  'requests': [],
  'frame_parent_id': None,
  'binary_questions': [],
  'compare_requests': []}]

## Part of interest

### Frame

In [11]:
frame=dataset['turns'][4][0]

### Text part

In [12]:
frame['text']

'Hello, I am looking to book a trip for 2 adults and 6 children for $21,300 or less. We are departing from Kochi for Denver.'

In [13]:
txt=frame['text']

### Entity part

In [14]:
frame['labels']['frames'][0]['info']

{'intent': [{'val': 'book', 'negated': False}],
 'budget': [{'val': '21.3', 'negated': False}],
 'dst_city': [{'val': 'Denver', 'negated': False}],
 'n_children': [{'val': '6', 'negated': False}],
 'or_city': [{'val': 'Kochi', 'negated': False}],
 'n_adults': [{'val': '2', 'negated': False}]}

In [15]:
entities=frame['labels']['frames'][0]['info']

In [16]:
entities.keys()

dict_keys(['intent', 'budget', 'dst_city', 'n_children', 'or_city', 'n_adults'])

In [17]:
### Entites we are interested in

In [18]:
pf_entities=['or_city','dst_city','str_date','end_date','budget']

# Functions for buildng the JSON

## Functions and tests

In [19]:
pf_entities=['or_city','dst_city','str_date','end_date','budget']

In [20]:
text=dataset['turns'][0][0]['text']

In [21]:
startIndex=text.find('book')
endCharIndex=text.find('book')+len('book')-1

In [69]:
def entityChar(entity_name,frame):
    #load entity part of the frame
    entities=frame['labels']['frames'][0]['info']
    str_entire=frame['labels']['frames'][0]['info'][entity_name][0]['val'].lower()
    return str_entire.split('.')[0]

def entityData(entity_char,text,entity_name):
    startIndex=max(text.find(entity_char),0)
    endCharIndex=max(text.find(entity_char)+len(entity_char)-1,0)
    return {'startCharIndex':startIndex,'endCharIndex':endCharIndex,'entityName':entity_name}

def entityDataDict(frame,pf_entities,intentName):
    text=frame['text'].lower()
    entities=frame['labels']['frames'][0]['info']    
    entity_list=[]
    for entity_name in pf_entities:
        #check if entities exist
        if entity_name in entities.keys():
            #check if value is not -1
            if entityChar(entity_name,frame)!='-1':
                entity_list.append(entityData(entityChar(entity_name,frame),text,entity_name))
    entityLabels_list=[
                    {'startCharIndex':0,
                     'endCharIndex':len(text),
                     'entityName':'flight',
                     'children':entity_list}
                    ]
    dict_output={
        "text":text,
        "intentName":intentName,
        "entityLabels":entity_list}
    
    
    
    
    return dict_output

In [177]:
entityDataDict(frame,pf_entities,"BookFlightIntent")

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intentName': 'BookFlightIntent',
 'entityLabels': [{'startCharIndex': 41,
   'endCharIndex': 47,
   'entityName': 'or_city'},
  {'startCharIndex': 27, 'endCharIndex': 34, 'entityName': 'dst_city'},
  {'startCharIndex': 62, 'endCharIndex': 70, 'entityName': 'str_date'},
  {'startCharIndex': 117, 'endCharIndex': 120, 'entityName': 'budget'}]}

In [71]:
utterances[0]

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intentName': 'BookFlightIntent',
 'entityLabels': [{'startCharIndex': 41,
   'endCharIndex': 47,
   'entityName': 'or_city'},
  {'startCharIndex': 27, 'endCharIndex': 34, 'entityName': 'dst_city'},
  {'startCharIndex': 62, 'endCharIndex': 70, 'entityName': 'str_date'},
  {'startCharIndex': 117, 'endCharIndex': 120, 'entityName': 'budget'}]}

In [68]:
utterances[0]

{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
 'intentName': 'BookFlightIntent',
 'entityLabels': [{'startCharIndex': 0,
   'endCharIndex': 122,
   'entityName': 'flight',
   'children': [{'startCharIndex': 41,
     'endCharIndex': 47,
     'entityName': 'or_city'},
    {'startCharIndex': 27, 'endCharIndex': 34, 'entityName': 'dst_city'},
    {'startCharIndex': 62, 'endCharIndex': 70, 'entityName': 'str_date'},
    {'startCharIndex': 117, 'endCharIndex': 120, 'entityName': 'budget'}]}]}

## Create utterances

In [70]:
utterances=[entityDataDict(dataset['turns'][x][0],pf_entities,"BookFlightIntent") for x in range(0,len(dataset['turns']))]

## train and test split

In [437]:
len(utterancesV2)*0.10

107.7

In [443]:
utterances_trainV2=utterancesV2[:-107]

In [444]:
utterances_testV2=utterancesV2[-107:]

In [445]:
len(utterances_testV2)

107

In [82]:
jsonString=json.dumps(utterances_test)

In [83]:
with open('C:/Users/Soyann/Code/Notebooks/OpenClassrooms/OC-P10-Chatbot/utterances_test.json', 'w') as outfile:
    for msg in utterances_test:        
        outfile.write(json.dumps(msg)+",\n")

In [446]:
len(utterances_trainV2)

970

# Create and train LUIS ressource

## Create variables

In [431]:
authoringKey = '1d1eab987e154e1f96503b0172d855d5'
authoringEndpoint = 'https://p10-authoring.cognitiveservices.azure.com/'
predictionKey = '384ea5f7c50c4c08a3212250b7cd1db4'
predictionEndpoint = 'https://p10.cognitiveservices.azure.com/'
appName = "P10 chatbot " + str(uuid.uuid4())
version_id = "0.1"
intentName = "BookFlightIntent"

In [432]:
appName

'P10 chatbot d7a5132a-4823-4968-99cf-f1de252eedda'

## Create the app

In [433]:
client = LUISAuthoringClient(authoringEndpoint, CognitiveServicesCredentials(authoringKey))

In [434]:
# define app basics
appDefinition = ApplicationCreateObject (name=appName, initial_version_id=version_id, culture='en-us')

# create app
app_id = client.apps.add(appDefinition)

# get app id - necessary for all other changes
print("Created LUIS app with ID {}".format(app_id))

Created LUIS app with ID a53e07f8-c3b6-4a8f-a387-12dc5f0d631b


In [435]:
# add new intention
client.model.add_intent(app_id, version_id, intentName)

'919382da-891b-48fd-8cbc-145f089153d3'

In [436]:
# add Entities
modelId = client.model.add_entity(app_id, version_id, name="or_city")
modelId = client.model.add_entity(app_id, version_id, name="dst_city")
modelId = client.model.add_entity(app_id, version_id, name="end_date")
modelId = client.model.add_entity(app_id, version_id, name="str_date")
modelId = client.model.add_entity(app_id, version_id, name="budget")

In [448]:
# Add examples and identify non conformity
unconformity=0
for x in range(0,len(utterances_trainV2)):
    print(x,end='')
    try:
        client.examples.add(app_id, version_id, utterances_trainV2[x])
    except Exception as e:
        unconformity+=1
        print(f'unconformity {unconformity} indice {x}')

0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258unconformity 1 indice 258
2592602612622632642652662672682692702712722732742752762772782792802812822832842852862872882892902912922932942952962972982993003013023033043053063073083093103113123133143153163173183193203213223233243253263273283293303313323333343353363373383393403413423433443453463473483493503513523533543553563573583593603

In [197]:
with open('C:/Users/Soyann/Code/Notebooks/OpenClassrooms/OC-P10-Chatbot/utterances_train_filtered.json', 'w') as outfile:
    outfile.write(json.dumps(utterances_train))

In [24]:
# Read the file previousy created
with open('C:/Users/Soyann/Code/Notebooks/OpenClassrooms/OC-P10-Chatbot/utterances_train_filtered.json') as f:
    utterances_train = json.load(f)

## Train the app

In [449]:
print("\nWe'll start training your app...")

async_training = client.train.train_version(app_id, version_id)
is_trained = async_training.status == "UpToDate"

trained_status = ["UpToDate", "Success"]
while not is_trained:
        time.sleep(1)
        status = client.train.get_status(app_id, version_id)
        is_trained = all(
        m.details.status in trained_status for m in status)


We'll start training your app...


## Publish the app

In [450]:
print("\nWe'll start publishing your app...")
client.apps.update_settings(app_id, is_public=True)
publish_result = client.apps.publish(app_id,version_id,is_staging=False)



We'll start publishing your app...


In [451]:
publish_result.as_dict()

{'version_id': '0.1',
 'is_staging': False,
 'endpoint_url': 'https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/a53e07f8-c3b6-4a8f-a387-12dc5f0d631b',
 'region': 'westeurope, northeurope, uksouth, ukwest, francecentral, francesouth, norwayeast, norwaywest',
 'endpoint_region': 'westeurope',
 'failed_regions': '',
 'published_date_time': '2021-12-22T17:15:02Z'}

In [452]:
endpoint = publish_result.endpoint_url + \
"?subscription-key=" + authoringKey + "&q="
print("Your app is published. You can now go to test it on\n{}".format(endpoint))

Your app is published. You can now go to test it on
https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/a53e07f8-c3b6-4a8f-a387-12dc5f0d631b?subscription-key=1d1eab987e154e1f96503b0172d855d5&q=


# Authenticate the prediction runtime client

In [453]:
runtimeCredentials = CognitiveServicesCredentials(predictionKey)
clientRuntime = LUISRuntimeClient(endpoint=predictionEndpoint, credentials=runtimeCredentials)

In [455]:
#test with one utterance
query = utterances_testV2[1]['text']

result = clientRuntime.prediction.resolve(app_id,query,verbose=False)

print("Detected entities:")
for entity in result.entities:
            print("\t-> Entity '{}' (type: {}, score:{:d}%)".format(
                entity.entity,
                entity.type,
                int(entity.additional_properties['score']*100)
            ))

Detected entities:
	-> Entity 'la' (type: or_city, score:92%)


## create a file for batch testing

In [419]:
def entityCharV2(entity_name,frame):
    #load entity part of the frame
    entities=frame['labels']['acts'][1]['args']
    for entity in entities:
        if entity['key']==entity_name:
            char=entity['val'].lower()
            if re.search(r"\b"+char+"\w+",frame['text'].lower()):
                print('catch')
                result=re.search(r"\b"+char+"\w+",frame['text'].lower()).group()
            else :
                result=char
    return result.split('.')[0]

def entityDataV2(entity_char,text,entity_name):
    startIndex=max(text.find(entity_char),0)
    endCharIndex=max(text.find(entity_char)+len(entity_char)-1,0)
    return {'startCharIndex':startIndex,'endCharIndex':endCharIndex,'entityName':entity_name}

def entityDataDictV2(frame,pf_entities,intentName):
    text=frame['text'].lower()
    entities=frame['labels']['acts'][1]['args']
    entities_keys=[entity['key'] for entity in entities]
    entity_list=[]
    for entity_name in pf_entities:
        #check if entities exist
        if entity_name in entities_keys:
            #check if value is not -1
            if entityCharV2(entity_name,frame)!='-1':
                entity_list.append(entityDataV2(entityCharV2(entity_name,frame),text,entity_name))
                    
    dict_output={
        "text":text,
        "intentName":intentName,
        "entityLabels":entity_list}
    
    return dict_output

In [458]:
A=[{'text':utterance['text'],
  'intent':utterance['intentName'],
  'entities':[{'entity':entity['entityName'],
               'startPos':entity['startCharIndex'],
               'endPos':entity['endCharIndex']
              } for entity in utterance['entityLabels']]
} for utterance in utterances_testV2]

In [459]:
len(A)

107

In [460]:
with open('C:/Users/Soyann/Code/Notebooks/OpenClassrooms/OC-P10-Chatbot/utterances_test_full.json', 'w') as outfile:
    outfile.write(json.dumps(A))